In [ ]:
import os.path as path
import os
import csv 
import json 
import pandas as pd
up = path.abspath(path.join('json_to_csv.ipynb' ,"../../../.."))

In [ ]:
up

In [ ]:
data_file = up + '/mkortas/detect-waste/annotations/annotat.csv'
json_path = up + '/TACO-master/data/annotations.json'
annot_folder = up + '/mkortas/detect-waste/annotations/'

In [ ]:
with open(json_path) as json_file: 
    data = json.load(json_file) 

In [ ]:
def export_data(data, dtype, file):
    img = data[dtype] 
    data_file = open(file, 'w') 
    csv_writer = csv.writer(data_file) 
    count = 0
    for i in img: 
        if count == 0: 
            header = i.keys() 
            csv_writer.writerow(header) 
            count += 1
        csv_writer.writerow(i.values()) 
    data_file.close() 

In [ ]:
export_data(data, 'images', os.path.join(annot_folder,'data_file.csv'))
export_data(data, 'annotations', os.path.join(annot_folder,'data_file2.csv'))
export_data(data, 'categories', os.path.join(annot_folder,'data_file3.csv'))

In [ ]:
data3 = pd.read_csv(os.path.join(annot_folder,'data_file3.csv'))
data2 = pd.read_csv(os.path.join(annot_folder,'data_file2.csv'))
data = pd.read_csv(os.path.join(annot_folder,'data_file.csv'))

In [ ]:
data = data.rename(columns={"id": "image_id"})
data3 = data3.rename(columns={"id": "category_id"})
data2 = data2.merge(data, on='image_id', how='left')
data2 = data2.merge(data3, on='category_id', how='left')
data2 = data2.rename(columns={"supercategory": "class"})
data2 = data2.rename(columns={"file_name": "filename"})
data2['bbox'] = data2['bbox'].map(lambda x: x.lstrip('[').rstrip(']'))
data2[['xmin','ymin','xmax','ymax']] = data2.bbox.str.split(",",expand=True,)

In [ ]:
data2['xmax'] = data2['xmax'].astype(float) + data2['width'].astype(float)
data2['ymax'] = data2['ymax'].astype(float) + data2['height'].astype(float)

In [ ]:
#data2['flickr_url'] = data2['flickr_url'].map(lambda x: x[-48:])
#data2['flickr_url'] = data2['flickr_url'].map(lambda x: x.replace('flickr.com/',''))
#data2[['cos','path']] = data2.flickr_url.str.split("/",expand=True,)
#data2 = data2[['path','width','height','class','xmin','ymin','xmax','ymax']]
#data2 = data2.rename(columns={"path": "filename"})

In [ ]:
data2 = data2[['filename','width','height','class','xmin','ymin','xmax','ymax']]
data2.to_csv(data_file,index=False)